In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
from torch.optim.lr_scheduler import StepLR

# Where the magic happens...
# torch.manual_seed(3456)

# Needed to download MNIST dataset without HTTP Error
from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

In [3]:
class MNIST_Net(nn.Module):
    def __init__(self, N=10):
        super(MNIST_Net, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1,  6, 5),
            nn.MaxPool2d(2, 2), # 6 24 24 -> 6 12 12
            nn.ReLU(True),
            nn.Conv2d(6, 16, 5), # 6 12 12 -> 16 8 8
            nn.MaxPool2d(2, 2), # 16 8 8 -> 16 4 4
            nn.ReLU(True)
        )
        self.classifier =  nn.Sequential(
            nn.Linear(16 * 4 * 4, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, N),
            nn.Softmax(1)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = x.view(-1, 16 * 4 * 4)
        x = self.classifier(x)
        return x

In [5]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
mnist_train_data = torchvision.datasets.MNIST(root='./MNIST', train=True, download=True,transform=transform)
mnist_test_data = torchvision.datasets.MNIST(root='./MNIST', train=False, download=True,transform=transform)

kwargs = {'batch_size': 1}

0.1%

100.1%

Extracting ./MNIST/MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/MNIST/raw


113.5%

Extracting ./MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/MNIST/raw


180.4%../torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


Extracting ./MNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/MNIST/raw
Extracting ./MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/MNIST/raw
Processing...
Done!


In [6]:
# ---------- train_data ----------
with open('train_data.txt') as f:
    train_data = f.readlines()
    
# Strip new lines
train_data = [d.strip() for d in train_data]

# Convert strings (e.g. "(datum_i, datum_j, sum)") to tuples of ints
train_data = [tuple(int(e) for e in d.strip("()").split(",")) for d in train_data]

# ---------- test data ----------
with open('test_data.txt') as f:
    test_data = f.readlines()
    
# Strip new lines
test_data = [d.strip() for d in test_data]

# Convert strings (e.g. "(datum_i, datum_j, sum)") to tuples of ints
test_data = [tuple(int(e) for e in d.strip("()").split(",")) for d in test_data]

# ---------- network and optimizer ----------
model = MNIST_Net()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [7]:
# Tensorize
train_data = torch.tensor(train_data)

# Batch
train_data = [train_data[i: i+ kwargs['batch_size']] for i in range(0, len(train_data), kwargs['batch_size'])]
train_data = torch.stack(train_data[:-1])

# Tensorize
test_data = torch.tensor(test_data)

# Batch
test_data = [test_data[i: i+kwargs['batch_size']] for i in range(0, len(test_data), kwargs['batch_size'])]
test_data = torch.stack(test_data[:-1])

In [8]:
def test():
    model.eval()

    total = 0
    correct = 0
    for j, test_batch in enumerate(test_data):
        idx1, idx2, summation = test_batch[0]
        X1 = mnist_test_data[idx1][0].unsqueeze(0)
        X2 = mnist_test_data[idx2][0].unsqueeze(0)

        output1 = model(X1)
        output2 = model(X2)

        pred1 = output1.argmax(dim=1, keepdim=False)
        pred2 = output2.argmax(dim=1, keepdim=False)
        correct += (summation == (pred1 + pred2)).sum()
        total += len(test_batch)

    print('Test Accuracy: {}/{} ({:.0f}%)\n'.format(correct, total, 100. * correct / total)) 

In [9]:
def brute_force(output1, output2, summation):
    combinations =  torch.cartesian_prod(torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))
    probs = []
    for combination in combinations:
        probs += [output1[0][combination[0]] * output2[0][combination[1]]]
    probs = torch.stack(probs)
    indices = (combinations[:, 0] + combinations[:, 1] == summation).nonzero(as_tuple=True)[0]
    return -torch.log(probs[indices].sum())

In [11]:
from tqdm import tqdm

NUM_EPOCHS = 1

for epoch in range(NUM_EPOCHS):
      
    # train
    for i, batch in enumerate(tqdm(train_data)):

        model.train()
        optimizer.zero_grad()
        
        idx1, idx2, summation = batch[0]
        X1 = mnist_train_data[idx1][0].unsqueeze(0)
        X2 = mnist_train_data[idx2][0].unsqueeze(0)
        output1 = model(X1)
        output2 = model(X2)
        
        pred1 = output1.argmax(dim=1, keepdim=False)
        pred2 = output2.argmax(dim=1, keepdim=False)
        
        closs = brute_force(output1, output2, summation)
            
        closs.backward()
        optimizer.step()
        
        if i % 1000 == 0 and i != 0:
            test()

  3%|▎         | 1018/29999 [00:17<1:08:58,  7.00it/s]

Test Accuracy: 3474/4999 (69%)



  7%|▋         | 2017/29999 [00:34<1:06:54,  6.97it/s]

Test Accuracy: 4388/4999 (88%)



 10%|█         | 3014/29999 [00:51<1:04:40,  6.95it/s]

...

 93%|█████████▎| 28011/29999 [08:33<05:12,  6.37it/s]

Test Accuracy: 4802/4999 (96%)



 97%|█████████▋| 29014/29999 [08:51<02:36,  6.28it/s]

Test Accuracy: 4815/4999 (96%)



100%|██████████| 29999/29999 [09:04<00:00, 55.07it/s]
